In [1]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*, value=None):
        self.feature = feature
        self.thershold = thershold
        self.left = left
        self.right = right
        self.value= None
        
        
    def is_leaf_node():
        return self.value is not None

In [4]:
import numpy as np
from collections import Counter
class Decision():
    def __init__(self, min_sample_split=2, max_depth=100, n_features=None):
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root=None
        
        
    def fit(self, x, y):
        self.n_features = x.shape[1] if not self.n_features else min(x.shape[1], self.n_features)
        self.root = self._grow_tree(x, y)
        
        
    def _grow_tree(self, x, y, depth):
        
        n_samples, n_feats = x.shape
        n_labels = np.unique(y)
        
        
        #check if we should stop
        if(depth>=self.max_depth or n_labels == 1 or n_samples<self.min_sample_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)
        #what's the best split
        best_feature, best_thresh = self._best_split(x, y, feat_idxs)
        
        
        #creat child nodes
        left_idx, right_idx = self._split(x[:,est_feature], best_thresh)
        left = self._grow_tree(x[left_idx, :], y[left_idx], depth=1)
        right = self._grow_tree(x[right_idx,:], y[right_idx], depth=1)
        
        return Node(best_feature, best_thresh, left, right)
        
        def _best_split(self, x, y, feat_idxs):
            best_gain = -1
            split_idx, split_threshold = None, None
            
            for feat_idx in feat_idxs:
                x_column = x[:, feat_idx]
                thresholds = np.unique(x_column)
                
                for thredhold in thresholds:
                    #finding out information gain
                    gain = self._info_gain(y, x_column, thredhold)
                    
                    if gain>best_gain:
                        best_gain = gain
                        split_idx = feat_idx
                        split_threshold = thredhold
                        
                        
            return split_idx, split_threshold
        
        
        def _info_gain(self, y, x_column, thredhold):
            
            #get entropy of parent: IG(Y,X) = E(Y) - E(Y|X)
            
            parent_entropy = self._entropy(y)
            
            #create children
            
            left_idx, right_idx = self._split(x_column, thredhold)
            
            if len(left_idx) == 0 or len(right_idx) == 0:
                return 0
            
            
            #finding the weight entropy for children
            n = len(y)
            n_l, n_r = len(left_idx), len(right_idx)
            e_l, e_r = self._entropy(y[left_idx]), self._entropy(y[right_idx])
            child_entropy = (n_l/n) * e_l + (n_r/n) * e_r
            
            
            # find info gain
            information_gain = parent_entropy - child_entropy
            return information_gain
            
        def _split(self, x_column, split_threshold):
            left_idx = np.argwhere(x_column<=split_threshold).flatten
            right_idx = np.argwhere(x_column>split_threshold).flatten
            return left_idx, right_idx
            
        def _entropy(self, y):
            hist = np.bincount(y)
            ps = hist / len(y) #len(y) is the same as p(x) in entropy formula
            return -np.sum([p * np.log(p) for p in ps if p > 0]) # - sum p*log(p)
        
        
        def _most_common_label(self, y):
            counter = Counter(y)
            value = counter.most_common[1][0][0]
            return value
        
    #now the training is done we need to predict    
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    
    
    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
        
        
        